<a href="https://colab.research.google.com/github/vpvinc/P7_loan_scoring_model/blob/main/P7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implémentez un modèle de scoring**

https://www.kaggle.com/c/home-credit-default-risk/data

This is a test 

another test

In [ ]:
# third test for commit with git bash
print('hello')

hello


test for git bash

test for colab